In [52]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister, ClassicalRegister
from qiskit.circuit.library import StatePreparation
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram, plot_state_city


In [ ]:
def int_to_bits(a, num_bits=None):
    """Decompose integer a into an array of bits (MSB first).
    If num_bits is specified, the result is zero-padded to that length."""
    bits = [(a >> i) & 1 for i in range(num_bits or a.bit_length())]
    return bits  # LSB first

def remove_leading_0s(bits):
    first_one = next((i for i, b in enumerate(bits) if b), len(bits))
    return bits[first_one:]

def add_classically_controlled_X(circuit, qs, bit):
    if bit:
        for q in qs:
            circuit.x(q)

def logical_and(qc, cq1, cq2, target):    
    qc.h(target)
    qc.t(target) 
    qc.cx(cq1, target) 
    qc.cx(cq2, target)
    qc.cx(target, cq1)
    qc.cx(target, cq2)
    qc.tdg(cq1) 
    qc.tdg(cq2)
    qc.t(target)
    qc.cx(target, cq1)
    qc.cx(target, cq2)
    qc.h(target)
    qc.s(target)
    
    return qc

def logical_and_dagger(qc, cq1, cq2, target):
    qc.sdg(target)
    qc.h(target)
    qc.cx(target, cq1)
    qc.cx(target, cq2)
    qc.t(cq1) 
    qc.t(cq2)
    qc.tdg(target)
    qc.cx(target, cq1)
    qc.cx(target, cq2) 
    qc.cx(cq2, target)
    qc.cx(cq1, target)
    qc.tdg(target)
    qc.h(target)
    
    return qc


def logical_and_uncompute(qc, cq1, cq2, target, clbit=None):
    if clbit is None:
        logical_and_dagger(qc, cq1, cq2, target)
        return
    
    qc.h(target)
    qc.measure(target, clbit)
    with qc.if_test((clbit, 1)):
        qc.cz(cq1, cq2)

    return qc


def addConstant(a, qubit_count, allow_measurement):
    if a == 0:
        return QuantumCircuit(qubit_count)
    
    a_bits = int_to_bits(a, num_bits=qubit_count)
    a_bits = remove_leading_0s(a_bits)
    bit_count = len(a_bits)
    offset = qubit_count - bit_count  # number of new LSB qubits to prepend

    if bit_count == 1:
        qc = QuantumCircuit(qubit_count)
        qc.x(qubit_count-1)
        return qc

    adderCircuit = addConstantClean(a_bits, bit_count, allow_measurement)

    # Build expanded circuit: full-width data register + same ancilla register
    new_data  = QuantumRegister(qubit_count, 'data')
    anc       = AncillaRegister(bit_count - 3, 'anc')
    qubit_map = [new_data[offset + i] for i in range(bit_count)] + list(anc)

    if allow_measurement:
        # Carry the single mid-circuit classical bit into the result circuit.
        # It lives in its own named register so it never collides with any
        # classical register added later (e.g. a final measurement register).
        mid    = ClassicalRegister(1, 'mid')
        result = QuantumCircuit(new_data, anc, mid,
                                name=f"+{int(''.join(map(str, a_bits)), 2)}")
        result.compose(adderCircuit, qubits=qubit_map, clbits=[mid[0]], inplace=True)
    else:
        result = QuantumCircuit(new_data, anc,
                                name=f"+{int(''.join(map(str, a_bits)), 2)}")
        result.compose(adderCircuit, qubits=qubit_map, inplace=True)

    return result

def addConstantClean(a_bits, qubit_count, allow_measurement):
    data = QuantumRegister(qubit_count, 'data')
    anc  = AncillaRegister(qubit_count - 3, 'anc')

    if allow_measurement:
        # One classical bit, reused by all four uncompute calls sequentially.
        mid   = ClassicalRegister(1, 'mid')
        qc    = QuantumCircuit(data, anc, mid,
                               name=f"+{int(''.join(map(str, a_bits)), 2)}")
        clbit = mid[0]
    else:
        qc    = QuantumCircuit(data, anc,
                               name=f"+{int(''.join(map(str, a_bits)), 2)}")
        clbit = None

    add_classically_controlled_X(qc, [data[0], data[1]], a_bits[1])
    logical_and(qc, data[0], data[1], anc[0])
    # qc.ccx(data[0], data[1], anc[0])

    add_classically_controlled_X(qc, [anc[0]], a_bits[1]^a_bits[2])
    add_classically_controlled_X(qc, [data[2]], a_bits[2])
    logical_and(qc, anc[0], data[2], anc[1])
    # qc.ccx(anc[0], data[2], anc[1])

    add_classically_controlled_X(qc, [anc[1]], a_bits[2]^a_bits[3])
    add_classically_controlled_X(qc, [data[3]], a_bits[3])
    logical_and(qc, anc[1], data[3], anc[2])
    # qc.ccx(anc[1], data[3], anc[2])

    add_classically_controlled_X(qc, [anc[2]], a_bits[3]^a_bits[4])
    add_classically_controlled_X(qc, [data[4]], a_bits[4])
    logical_and(qc, anc[2], data[4], anc[3])
    # qc.ccx(anc[2], data[4], anc[3])

    add_classically_controlled_X(qc, [anc[3]], a_bits[4]^a_bits[5])
    add_classically_controlled_X(qc, [data[5]], a_bits[5])
    qc.ccx(anc[3], data[5], data[6])

    add_classically_controlled_X(qc, [anc[3]], a_bits[5])

    qc.cx(anc[3], data[5])
    add_classically_controlled_X(qc, [anc[3]], a_bits[4])

    logical_and_uncompute(qc, anc[2], data[4], anc[3], clbit)
    # qc.ccx(anc[2], data[4], anc[3])
    add_classically_controlled_X(qc, [anc[2]], a_bits[4])

    qc.cx(anc[2], data[4])
    add_classically_controlled_X(qc, [anc[2]], a_bits[3])

    logical_and_uncompute(qc, anc[1], data[3], anc[2], clbit)
    # qc.ccx(anc[1], data[3], anc[2])
    add_classically_controlled_X(qc, [anc[1]], a_bits[3])

    qc.cx(anc[1], data[3])
    add_classically_controlled_X(qc, [anc[1]], a_bits[2])

    logical_and_uncompute(qc, anc[0], data[2], anc[1], clbit)
    # qc.ccx(anc[0], data[2], anc[1])
    add_classically_controlled_X(qc, [anc[0]], a_bits[2])

    qc.cx(anc[0], data[2])
    add_classically_controlled_X(qc, [anc[0]], a_bits[1])

    logical_and_uncompute(qc, data[0], data[1], anc[0], clbit)
    # qc.ccx(data[0], data[1], anc[0])
    add_classically_controlled_X(qc, [data[0]], a_bits[0])

    qc.cx(data[0], data[1])

    qc.x(data[0])
    add_classically_controlled_X(qc, [data[6]], a_bits[5]^a_bits[6])

    return qc


In [54]:
addConstant(150, 8, True).draw()

»
data_0: ─────────────────────────────────────────────────────────────────────»
        ┌───┐               ┌───┐┌─────┐       ┌───┐                         »
data_1: ┤ X ├───────■───────┤ X ├┤ Tdg ├───────┤ X ├─────────────────────────»
        ├───┤       │       └─┬─┘└┬───┬┘┌─────┐└─┬─┘┌───┐                    »
data_2: ┤ X ├───────┼────■────┼───┤ X ├─┤ Tdg ├──┼──┤ X ├────────────────────»
        └───┘       │    │    │   └─┬─┘ └─────┘  │  └─┬─┘                    »
data_3: ────────────┼────┼────┼─────┼────────────┼────┼──────────────────────»
        ┌───┐       │    │    │     │            │    │                      »
data_4: ┤ X ├───────┼────┼────┼─────┼────────────┼────┼──────────────────────»
        └───┘       │    │    │     │            │    │                      »
data_5: ────────────┼────┼────┼─────┼────────────┼────┼──────────────────────»
                    │    │    │     │            │    │                      »
data_6: ────────────┼────┼────┼─────┼────────────┼────┼──────────────────────»
                    │    │    │     │            │    │                      »
data_7: ────────────┼────┼────┼─────┼────────────┼────┼──────────────────────»
        ┌───┐┌───┐┌─┴─┐┌─┴─┐  │     │    ┌───┐   │    │  ┌───┐┌───┐┌───┐     »
 anc_0: ┤ H ├┤ T ├┤ X ├┤ X ├──■─────■────┤ T ├───■────■──┤ H ├┤ S ├┤ X ├──■──»
        ├───┤├───┤└───┘└───┘             └───┘           └───┘└───┘└───┘┌─┴─┐»
 anc_1: ┤ H ├┤ T ├──────────────────────────────────────────────────────┤ X ├»
        ├───┤├───┤                                                      └───┘»
 anc_2: ┤ H ├┤ T ├───────────────────────────────────────────────────────────»
        ├───┤├───┤                                                           »
 anc_3: ┤ H ├┤ T ├───────────────────────────────────────────────────────────»
        └───┘└───┘                                                           »
 mid: 1/═════════════════════════════════════════════════════════════════════»
                                                                             »
«                                                                           »
«data_0: ───────────────────────────────────────────────────────────────────»
«                                                                           »
«data_1: ───────────────────────────────────────────────────────────────────»
«                                                                           »
«data_2: ───────────────────────────────────────────────────────────────────»
«                  ┌───┐┌─────┐     ┌───┐                                   »
«data_3: ──■───────┤ X ├┤ Tdg ├─────┤ X ├───────────────────────────────────»
«          │       └─┬─┘└─────┘     └─┬─┘                              ┌───┐»
«data_4: ──┼─────────┼────────────────┼────────────────────────■───────┤ X ├»
«          │         │                │                        │       └─┬─┘»
«data_5: ──┼─────────┼────────────────┼────────────────────────┼─────────┼──»
«          │         │                │                        │         │  »
«data_6: ──┼─────────┼────────────────┼────────────────────────┼─────────┼──»
«          │         │                │                        │         │  »
«data_7: ──┼─────────┼────────────────┼────────────────────────┼─────────┼──»
«          │  ┌───┐  │  ┌─────┐┌───┐  │                        │         │  »
« anc_0: ──┼──┤ X ├──┼──┤ Tdg ├┤ X ├──┼────────────────────────┼─────────┼──»
«        ┌─┴─┐└─┬─┘  │  └┬───┬┘└─┬─┘  │  ┌───┐┌───┐┌───┐       │  ┌───┐  │  »
« anc_1: ┤ X ├──■────■───┤ T ├───■────■──┤ H ├┤ S ├┤ X ├──■────┼──┤ X ├──┼──»
«        └───┘           └───┘           └───┘└───┘└───┘┌─┴─┐┌─┴─┐└─┬─┘  │  »
« anc_2: ───────────────────────────────────────────────┤ X ├┤ X ├──■────■──»
«                                                       └───┘└───┘          »
« anc_3: ───────────────────────────────────────────────────────────────────»
«                                                                       

In [55]:
from qiskit.quantum_info import Statevector

def test_addConstant(a, qubit_count, test_inputs):
    """
    For each x in test_inputs, initialise the data register to |x>,
    apply addConstant(a, qubit_count), and verify the data register
    holds |(x + a) mod 2^qubit_count>.
    Ancilla qubits are initialised to |0> and must return to |0>.
    """
    circ = addConstant(a, qubit_count, allow_measurement=False)
    n_data = qubit_count
    n_anc  = circ.num_qubits - n_data
    results = []

    for x in test_inputs:
        # Build an initialisation circuit for data+ancilla
        init = QuantumCircuit(circ.num_qubits)
        for i in range(n_data):
            if (x >> i) & 1:
                init.x(i)          # data qubits are the first n_data

        full = init.compose(circ)
        sv   = Statevector(full)

        # Find the computational basis state with amplitude 1
        probs = sv.probabilities_dict()
        measured = max(probs, key=probs.get)   # bitstring, MSB first

        # Split into ancilla (high bits) and data (low bits)
        anc_bits  = measured[:n_anc]
        data_bits = measured[n_anc:]           # LSB of data is rightmost char

        data_val = int(data_bits, 2)
        expected = (x + a) % (2 ** n_data)
        ok = (data_val == expected) and (anc_bits == '0' * n_anc)
        results.append((x, data_val, expected, ok))
        print(f"x={x:3d}  got={data_val:3d}  expected={expected:3d}  ancilla_clean={anc_bits == '0'*n_anc}  {'OK' if ok else 'FAIL'}")

    all_ok = all(r[3] for r in results)
    print(f"\nAll tests passed: {all_ok}")
    return all_ok

# Test addConstant(150, 8) on several input values
test_addConstant(150, 8, test_inputs=[0, 1, 2, 5, 10, 50, 100, 105])


x=  0  got=150  expected=150  ancilla_clean=True  OK
x=  1  got=151  expected=151  ancilla_clean=True  OK
x=  2  got=152  expected=152  ancilla_clean=True  OK
x=  5  got=155  expected=155  ancilla_clean=True  OK
x= 10  got=160  expected=160  ancilla_clean=True  OK
x= 50  got=200  expected=200  ancilla_clean=True  OK
x=100  got=250  expected=250  ancilla_clean=True  OK
x=105  got=255  expected=255  ancilla_clean=True  OK

All tests passed: True


True

In [56]:
def addConstantClean_dagger(a_bits, qubit_count):
    # allow_measurement=False: dagger must be unitary for .inverse() to work
    return addConstantClean(a_bits, qubit_count, allow_measurement=False).inverse()

def addConstant_dagger(a, qubit_count):
    # allow_measurement=False: dagger must be unitary for .inverse() to work
    return addConstant(a, qubit_count, allow_measurement=False).inverse()


In [57]:

from qiskit.quantum_info import Statevector

def test_logical_and():
    """Verify logical_and computes |c1,c2,0> -> |c1,c2,c1 AND c2>
    and logical_and_dagger inverts it."""
    print("=== logical_and ===")
    results = []
    for c1 in [0, 1]:
        for c2 in [0, 1]:
            qc = QuantumCircuit(3)
            if c1: qc.x(0)
            if c2: qc.x(1)
            logical_and(qc, 0, 1, 2)
            sv = Statevector(qc)
            probs = sv.probabilities_dict()
            measured = max(probs, key=probs.get)
            t_out  = int(measured[0])
            c2_out = int(measured[1])
            c1_out = int(measured[2])
            expected_t = c1 & c2
            ok = (t_out == expected_t) and (c1_out == c1) and (c2_out == c2)
            results.append(ok)
            print(f"  c1={c1} c2={c2} -> target={t_out} (exp {expected_t}), ctrls=({c1_out},{c2_out})  {'OK' if ok else 'FAIL'}")

    print("\n=== logical_and then logical_and_dagger = identity ===")
    for c1 in [0, 1]:
        for c2 in [0, 1]:
            qc = QuantumCircuit(3)
            if c1: qc.x(0)
            if c2: qc.x(1)
            logical_and(qc, 0, 1, 2)
            logical_and_dagger(qc, 0, 1, 2)
            sv = Statevector(qc)
            probs = sv.probabilities_dict()
            measured = max(probs, key=probs.get)
            back_ok = (int(measured[2]) == c1 and int(measured[1]) == c2 and int(measured[0]) == 0)
            results.append(back_ok)
            print(f"  c1={c1} c2={c2}: returned to |c1,c2,0> = {'OK' if back_ok else 'FAIL'}")

    print(f"\nAll tests passed: {all(results)}")

test_logical_and()


=== logical_and ===
  c1=0 c2=0 -> target=0 (exp 0), ctrls=(0,0)  OK
  c1=0 c2=1 -> target=0 (exp 0), ctrls=(0,1)  OK
  c1=1 c2=0 -> target=0 (exp 0), ctrls=(1,0)  OK
  c1=1 c2=1 -> target=1 (exp 1), ctrls=(1,1)  OK

=== logical_and then logical_and_dagger = identity ===
  c1=0 c2=0: returned to |c1,c2,0> = OK
  c1=0 c2=1: returned to |c1,c2,0> = OK
  c1=1 c2=0: returned to |c1,c2,0> = OK
  c1=1 c2=1: returned to |c1,c2,0> = OK

All tests passed: True


In [58]:
def test_addConstant_dagger(a, qubit_count, test_inputs):
    """
    Two checks:
    1. addConstant_dagger alone computes |x> -> |(x - a) mod 2^n>
    2. addConstant followed by addConstant_dagger returns to |x> (identity)
    """
    circ_fwd = addConstant(a, qubit_count, allow_measurement=False)
    circ_dag = addConstant_dagger(a, qubit_count)
    n_anc = circ_fwd.num_qubits - qubit_count

    print(f"=== addConstant_dagger({a}, {qubit_count}) — subtraction ===")
    sub_ok = True
    for x in test_inputs:
        init = QuantumCircuit(circ_dag.num_qubits)
        for i in range(qubit_count):
            if (x >> i) & 1:
                init.x(i)
        sv = Statevector(init.compose(circ_dag))
        measured = max(sv.probabilities_dict(), key=sv.probabilities_dict().get)
        anc_bits  = measured[:n_anc]
        data_val  = int(measured[n_anc:], 2)
        expected  = (x - a) % (2 ** qubit_count)
        ok = (data_val == expected) and (anc_bits == '0' * n_anc)
        sub_ok &= ok
        print(f"  x={x:3d}  got={data_val:3d}  expected={expected:3d}  ancilla_clean={anc_bits == '0'*n_anc}  {'OK' if ok else 'FAIL'}")

    print(f"\n=== addConstant then addConstant_dagger — identity ===")
    id_ok = True
    for x in test_inputs:
        init = QuantumCircuit(circ_fwd.num_qubits)
        for i in range(qubit_count):
            if (x >> i) & 1:
                init.x(i)
        sv = Statevector(init.compose(circ_fwd).compose(circ_dag))
        measured = max(sv.probabilities_dict(), key=sv.probabilities_dict().get)
        anc_bits = measured[:n_anc]
        data_val = int(measured[n_anc:], 2)
        ok = (data_val == x) and (anc_bits == '0' * n_anc)
        id_ok &= ok
        print(f"  x={x:3d}  returned={data_val:3d}  ancilla_clean={anc_bits == '0'*n_anc}  {'OK' if ok else 'FAIL'}")

    all_ok = sub_ok and id_ok
    print(f"\nAll tests passed: {all_ok}")
    return all_ok

test_addConstant_dagger(150, 8, test_inputs=[0, 1, 2, 5, 10, 50, 100, 150, 200, 255])


=== addConstant_dagger(150, 8) — subtraction ===
  x=  0  got=106  expected=106  ancilla_clean=True  OK
  x=  1  got=107  expected=107  ancilla_clean=True  OK
  x=  2  got=108  expected=108  ancilla_clean=True  OK
  x=  5  got=111  expected=111  ancilla_clean=True  OK
  x= 10  got=116  expected=116  ancilla_clean=True  OK
  x= 50  got=156  expected=156  ancilla_clean=True  OK
  x=100  got=206  expected=206  ancilla_clean=True  OK
  x=150  got=  0  expected=  0  ancilla_clean=True  OK
  x=200  got= 50  expected= 50  ancilla_clean=True  OK
  x=255  got=105  expected=105  ancilla_clean=True  OK

=== addConstant then addConstant_dagger — identity ===
  x=  0  returned=  0  ancilla_clean=True  OK
  x=  1  returned=  1  ancilla_clean=True  OK
  x=  2  returned=  2  ancilla_clean=True  OK
  x=  5  returned=  5  ancilla_clean=True  OK
  x= 10  returned= 10  ancilla_clean=True  OK
  x= 50  returned= 50  ancilla_clean=True  OK
  x=100  returned=100  ancilla_clean=True  OK
  x=150  returned=150  

True